In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import time
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from cuml import RandomForestClassifier, LogisticRegression, SVC, LinearSVC
from xgboost import XGBClassifier
from cuml.manifold import UMAP
from skorch import NeuralNetClassifier
from imblearn.over_sampling import SMOTE
from sklearn.inspection import permutation_importance
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  #Shows all columns
pd.set_option('display.max_rows', None)     #Shows all rows
pd.set_option('display.max_colwidth', None) #Shows full content of each column

#Load the dataset
file_path = r'dataset.csv'
df = pd.read_csv(file_path)

#Display basic information about the dataset
print("Basic Dataset Information:")
print(df.info())

#Display summary statistics
print("\nSummary Statistics:")
print(df.describe())


In [ ]:
#Let's view all the variables names and data types
print("Column Name and Data Type:")
for column in df.columns:
    print(f"{column}: {df[column].dtype}")

In [ ]:
#Create lists of my numerical and categorical features.  Created list of IDs for use in excluding them from analysis
numerical_features = [
    'age', 'bmi', 'height', 'weight',
    'pre_icu_los_days',
    'apache_2_diagnosis', 'apache_3j_diagnosis',
    'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob',
    'arf_apache',
    'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache', 'gcs_unable_apache',
    'heart_rate_apache', 'map_apache', 'resprate_apache', 'temp_apache',
    'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min',
    'd1_heartrate_max', 'd1_heartrate_min',
    'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min',
    'd1_resprate_max', 'd1_resprate_min',
    'd1_spo2_max', 'd1_spo2_min',
    'd1_sysbp_max', 'd1_sysbp_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min',
    'd1_temp_max', 'd1_temp_min',
    'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min',
    'h1_heartrate_max', 'h1_heartrate_min',
    'h1_mbp_max', 'h1_mbp_min', 'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min',
    'h1_resprate_max', 'h1_resprate_min',
    'h1_spo2_max', 'h1_spo2_min',
    'h1_sysbp_max', 'h1_sysbp_min', 'h1_sysbp_noninvasive_max', 'h1_sysbp_noninvasive_min',
    'd1_glucose_max', 'd1_glucose_min',
    'd1_potassium_max', 'd1_potassium_min'
]

categorical_features = [
    'ethnicity', 'gender', 'icu_admit_source', 'icu_stay_type', 'icu_type',
    'elective_surgery', 'apache_post_operative',
    'intubated_apache', 'ventilated_apache',
    'apache_3j_bodysystem', 'apache_2_bodysystem',
    'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure', 'immunosuppression',
    'leukemia', 'lymphoma', 'solid_tumor_with_metastasis'
]

identifier_columns = [
    'encounter_id', 'patient_id', 'hospital_id', 'icu_id'
]

In [ ]:
#Visualize the distributions of each numerical feature
print("\nVisualizing Distributions of Numerical Features:")
for column in numerical_features:
    plt.figure(figsize=(10, 4))
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()

#Visualize the distribution of each categorical feature
print("\nVisualizing Distributions of Categorical Features:")
for column in categorical_features:
    plt.figure(figsize=(10, 4))
    sns.countplot(x=column, data=df)
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=45)
    plt.show()
    
#Show distribution of the target variable as well
column = "hospital_death"
plt.figure(figsize=(12, 6))  
sns.countplot(x=column, data=df)
plt.title(f'Distribution of {column}')
plt.xticks(rotation=45)
plt.tight_layout()  #Adjust layout to make room for title and labels
plt.show()

In [ ]:
# Calculate skewness and kurtosis for each column in the DataFrame to assess normality of each distribution
for column in numerical_features:
    skewness = skew(df[column].dropna())  #dropna to ignore missing values
    kurt = kurtosis(df[column].dropna())
    print(f"Skewness of {column}: {skewness:.2f}")
    print(f"Kurtosis of {column}: {kurt:.2f}")


In [ ]:
#Highlight class imbalance
df.groupby(['hospital_death']).count()

In [ ]:
#Handling missing values
print("\nHandling Missing Values:")
missing_data = df.isnull().sum()
print("Missing data in each column:\n", missing_data)

#Calculate the percentage of missing data in each column
missing_data = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_data / len(df)) * 100

#Filter out columns that have no missing data to clean up visualization
missing_percentage = missing_percentage[missing_percentage > 0]

#Print the percentage of missing data
print("Percentage of missing data per column:\n")
print(missing_percentage)

#Visualizing the missing data percentages
plt.figure(figsize=(15, 8))
sns.barplot(x=missing_percentage.index, y=missing_percentage)
plt.ylabel('Percentage of Missing Data')
plt.xlabel('Columns')
plt.title('Percentage of Missing Data by Feature')
plt.xticks(rotation=90) 
plt.show()

In [ ]:
#Test for MCAR across all variable pairs
def test_mcar(df):
    """Test for MCAR by checking if patterns of missingness are correlated across pairs of variables."""
    missing_indicators = df.isnull().astype(int)
    cols = df.columns
    mcar_vars = []

    for col in cols:
        p_values = []
        for other_col in cols:
            if col != other_col:
                contingency_table = pd.crosstab(missing_indicators[col], missing_indicators[other_col])
                _, p, _, _ = chi2_contingency(contingency_table)
                p_values.append(p)
        
        #If all p-values are high, we assume MCAR for this variable
        if all(p > 0.05 for p in p_values):
            mcar_vars.append(col)

    return mcar_vars

def analyze_missingness(df):
    """Identify and classify the type of missing data in each variable."""
    #First, determine MCAR variables
    mcar_vars = test_mcar(df)
    mar_mnar_vars = [col for col in df.columns if col not in mcar_vars and df[col].isnull().any()]

    #Assume remaining variables could be MAR or MNAR - usually requires deeper analysis or domain knowledge
    mar_vars, mnar_vars = mar_mnar_vars, []

    # Output results
    print("Variables likely MCAR:", mcar_vars)
    print("Variables potentially MAR or MNAR:", mar_vars)
    
analyze_missingness(df)

In [ ]:
#Generate a clean dataset
#Let's proceed with dropping the Unnamed: 83 column
df = df.drop(columns=['Unnamed: 83'])
df_original = df.copy(deep=True)

In [ ]:
#Now let's perform imputation utilizing Medeian and Most Frequent for numerical and categorical features respectively
numerical_imputer = SimpleImputer(strategy='median')
df[numerical_features] = numerical_imputer.fit_transform(df[numerical_features])
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_features] = categorical_imputer.fit_transform(df[categorical_features])
df_imputed = df

In [ ]:
###Let's now show that imputation didn't modify the data distribution in a critical manner
#Create a large figure to hold subplots
plt.figure(figsize=(15, 85))

#Create separate plots for each feature
for column in numerical_features:
    plt.figure(figsize=(10, 5))
    sns.histplot(df_original[column].dropna(), kde=True, color='blue', label='Before')
    sns.histplot(df_imputed[column], kde=True, color='red', alpha=0.6, label='After')
    plt.title(f'Numerical - {column}')
    plt.legend()
    plt.show()

for column in categorical_features:
    plt.figure(figsize=(10, 5))
    sns.countplot(y=column, data=df_original, color='blue', label='Before', order=df_original[column].value_counts().index)
    sns.countplot(y=column, data=df_imputed, color='red', alpha=0.6, label='After', order=df_imputed[column].value_counts().index)
    plt.title(f'Categorical - {column}')
    plt.legend()
    plt.show()

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Calculate correlation matrix for numerical features only
numerical_df = df_imputed.select_dtypes(include=[np.number])
correlation_matrix = numerical_df.corr()

In [ ]:
#Calculate correlation matrix for numerical features only
numerical_df = df_imputed.select_dtypes(include=[np.number])
correlation_matrix = numerical_df.corr()
# Set the default display size for heatmaps
sns.set(rc={'figure.figsize':(20,20)})

#Generate the full correlation matrix heatmap
plt.figure(figsize=(20, 20))
print(correlation_matrix.shape)
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Full Correlation Matrix')
plt.show()




In [ ]:
#Identifying variables with high collinearity
print("\nIdentifying Highly Correlated Variables:")
threshold = 0.8  # You can adjust this threshold
highly_correlated_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold and (correlation_matrix.columns[j], correlation_matrix.columns[i]) not in highly_correlated_pairs:
            highly_correlated_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))

if not highly_correlated_pairs:
    print("No highly correlated pairs found.")
else:
    for pair in highly_correlated_pairs:
        print(f"{pair[0]} and {pair[1]} have a correlation coefficient higher than {threshold}.")

In [ ]:
#List of variables to drop due to high correlation
variables_to_drop = [
    'weight', 'gcs_eyes_apache', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min',
    'heart_rate_apache', 'd1_diasbp_max', 'd1_mbp_min', 'd1_diasbp_min',
    'd1_mbp_max', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'h1_diasbp_noninvasive_max',
    'h1_diasbp_noninvasive_min', 'h1_heartrate_max', 'h1_diasbp_max', 'h1_mbp_min',
    'h1_diasbp_min', 'h1_mbp_max', 'h1_sysbp_noninvasive_max', 'h1_mbp_noninvasive_min',
    'h1_sysbp_noninvasive_min', 'apache_4a_hospital_death_prob'
]

#Dropping highly correlated variables
df_imputed.drop(columns=variables_to_drop, inplace=True)

#Ensure that the features to standardize are still in the dataframe after dropping correlated ones
numerical_features = [feature for feature in numerical_features if feature in df_imputed.columns]

In [ ]:
#Create a new copy of imputed data frame for manipulation to save from having to re-run imputation again
df_cleaned = df_imputed.copy(deep=True)

In [ ]:
def find_outliers_zscore(df, features, threshold=2.5):
    z_scores = np.abs(stats.zscore(df[features]))
    mask = (z_scores < threshold).all(axis=1)
    return df[mask], df[~mask]

#Adjust the threshold and detect outliers
#Value threshold of 10 used here as lower values resulted in large chunks (over 50% or more of the dataset being throw out)
outlier_free_df, outliers_df = find_outliers_zscore(df_cleaned, numerical_features, threshold=10)


In [ ]:
#Number of observations remaining
len(outlier_free_df)

# EXPERIMENT 1 - All Features + No Dim Reduction + Class Imbalance

In [ ]:
###NO dimensionality reduction + no handling of class imbalance

df_cleaned = outlier_free_df.copy(deep=True)  # Assuming outlier_free_df is defined

#Define the percentage of the data to use
data_percentage = 1
df_sampled = df_cleaned.sample(frac=data_percentage, random_state=42)

print(f"Number of samples used: {len(df_sampled)}")

X = df_sampled.drop('hospital_death', axis=1)
y = df_sampled['hospital_death'].astype(np.float32)


#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

#Apply preprocessing and convert to numpy arrays for compatibility with cuML and XGBoost
X_train_processed = preprocessor.fit_transform(X_train).astype(np.float32)
X_test_processed = preprocessor.transform(X_test).astype(np.float32)

#We need to count the number of features after transformation
num_features = X_train_processed.shape[1]

#Define the PyTorch model
class SimpleNN(nn.Module):
    def __init__(self, num_features, num_units=128, activation=nn.ReLU()):
        super(SimpleNN, self).__init__()
        self.dense1 = nn.Linear(num_features, num_units)
        self.activation = activation
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(num_units, 1)  #Output layer for binary classification

    def forward(self, X):
        X = self.activation(self.dense1(X))
        X = self.dropout(X)
        X = self.output(X)
        return X.squeeze()  #Ensure the output is of shape [batch_size]

#Neural network settings
net = NeuralNetClassifier(
    SimpleNN,
    module__num_features=num_features, 
    criterion=nn.BCEWithLogitsLoss,
    optimizer=torch.optim.Adam,
    max_epochs=10,
    batch_size=64,
    device='cuda',
    iterator_train__shuffle=True
)


#Dictionary to store models and their grid search parameters
models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=2000),
        'params': {
            'C': [0.1, 1, 10, 100]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 15, 20]
        }
    },
    'SVM': {
        'model': SVC(kernel='linear', cache_size=2000),
        'params': {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
        }
    },
    'XGBoost': {
        'model': XGBClassifier(tree_method='hist', device='cuda'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.1, 0.01, 0.001]
        }
    }
}

#Dictionary to store best models and scores
best_models = {}

#Run grid search for each model
for model_name, info in models_params.items():
    clf = GridSearchCV(info['model'], info['params'], cv=5, scoring='accuracy', n_jobs=1)
    clf.fit(X_train_processed, y_train)
    best_models[model_name] = {
        'model': clf.best_estimator_,
        'score': clf.best_score_,
        'best_params': clf.best_params_
    }
    print(f"{model_name}: Best CV score - {clf.best_score_:.2f}")
    print(f"Best parameters for {model_name}: {clf.best_params_}")

#Setup and run GridSearchCV specifically for the neural network
nn_params = {
    'lr': [0.1, 0.01, 0.001],
    'max_epochs': [10, 20, 30, 50, 100],
    'batch_size': [64, 128],
    'module__num_units': [100, 200],
    'module__activation': [nn.ReLU(), nn.Tanh()]
}
gs_nn = GridSearchCV(net, nn_params, refit=True, cv=5, scoring='accuracy', verbose=0)
gs_nn.fit(X_train_processed, y_train)
best_models['PyTorch NN'] = {
    'model': gs_nn.best_estimator_,
    'score': gs_nn.best_score_,
    'best_params': gs_nn.best_params_
}
print("PyTorch NN: Best CV score - {:.2f}".format(gs_nn.best_score_))
print("Best parameters for PyTorch NN: ", gs_nn.best_params_)

#Evaluate all models on the test set
test_scores = {}
for model_name, info in best_models.items():
    test_score = info['model'].score(X_test_processed, y_test)
    test_scores[model_name] = test_score
    print(f"Test Score for {model_name}: {test_score:.2f}")

#Visualization of the performance of the best model for each type
fig, ax = plt.subplots()
model_names = list(best_models.keys())
scores = [test_scores[m] for m in model_names]
ax.barh(model_names, scores, color='skyblue')
ax.set_xlabel('Accuracy on Test Set')
ax.set_title('Model Performance Comparison on Test Data')
plt.show()

In [ ]:
#Evaluate all models on the test set and generate precision and recall values and 
#visualize cross validation and test score results
test_scores = {}
additional_metrics = {}
for model_name, info in best_models.items():
    model = info['model']
    y_pred = model.predict(X_test_processed)
    test_scores[model_name] = model.score(X_test_processed, y_test)
    #y_prob = model.decision_function(X_test_umap)
    #auc_score = roc_auc_score(y_test, y_prob)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    additional_metrics[model_name] = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1#,
        #'auc': auc_score
    }
    print(f"Test Score for {model_name}: {test_scores[model_name]:.9f}")
    print(f"Precision for {model_name}: {precision:.9f}")
    print(f"Recall for {model_name}: {recall:.9f}")
    print(f"F1 Score for {model_name}: {f1:.9f}")
    #print(f"AUC Score for {model_name}: {auc_score:.9f}")

fig, ax = plt.subplots(figsize=(10, 4))
model_names = list(best_models.keys())
scores = [best_models[m]['score'] for m in model_names]
bar_height = 0.4
ax.barh(model_names, scores, height=bar_height, color='skyblue')
ax.set_xlabel('Best Cross-Validation Accuracy')
ax.set_title('Model Comparison')
plt.show()

fig, ax = plt.subplots(figsize=(10, 4))
scores = [test_scores[m] for m in model_names]
ax.barh(model_names, scores, height=bar_height, color='skyblue')
ax.set_xlabel('Accuracy on Test Set')
ax.set_title('Model Performance Comparison on Test Data')
plt.show()

In [ ]:
#Generate Feature Importances
feature_importances = {}
for model_name, model_info in best_models.items():
    model = model_info['model']
    if hasattr(model, 'feature_importances_'):
        feature_importances[model_name] = model.feature_importances_
    elif hasattr(model, 'coef_'):
        feature_importances[model_name] = np.abs(model.coef_)

#Neural Network Permutation Importance
nn_model = best_models['PyTorch NN']['model']
perm = permutation_importance(nn_model, X_test_processed, y_test, n_repeats=5, random_state=42)
feature_importances['PyTorch NN'] = perm.importances_mean

for model_name, importances in feature_importances.items():
    print(f"Feature importances for {model_name}: {importances}")

# EXPERIMENT 2 - UMAP + SMOTE

In [ ]:
###UMAP + SMOTE 
df_cleaned = outlier_free_df.copy(deep=True)  

#Define the percentage of the data to use
data_percentage = 1
df_sampled = df_cleaned.sample(frac=data_percentage, random_state=42)

print(f"Number of samples used: {len(df_sampled)}")

X = df_sampled.drop('hospital_death', axis=1)
y = df_sampled['hospital_death'].astype(np.float32)

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

#Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

#Apply SMOTE
smote = SMOTE(random_state=42)
X_train_processed, y_train = smote.fit_resample(X_train_processed, y_train)


#Initialize and fit UMAP
umap_reducer = UMAP(n_components=3, n_neighbors=15, random_state=42)
X_train_umap = umap_reducer.fit_transform(X_train_processed.astype(np.float32))
X_test_umap = umap_reducer.transform(X_test_processed.astype(np.float32))

#Define models and parameters for GridSearchCV
models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=2000),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200, 400],
            'max_depth': [10, 15, 20, 50]
        }
    },
    'Linear SVC': {
        'model': LinearSVC(),
        'params': {
            'C': [0.1, 1, 10, 100]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(tree_method='hist', device='cuda'),
        'params': {
            'n_estimators': [50, 100, 200, 500],
            'max_depth': [3, 6, 9, 15, 30],
            'learning_rate': [0.1, 0.01, 0.001]
        }
    }
    
}

#Dictionary to store best models and scores
best_models = {}

#Run grid search for each model
for model_name, info in models_params.items():
    start_time = time.time()
    clf = GridSearchCV(info['model'], info['params'], cv=5, scoring='accuracy', n_jobs=1)
    clf.fit(X_train_umap, y_train)
    end_time = time.time()
    best_models[model_name] = {
        'model': clf.best_estimator_,
        'score': clf.best_score_,
        'best_params': clf.best_params_,
        'training_time': end_time - start_time
    }
    print(f"{model_name}: Best CV score - {clf.best_score_:.9f}")
    print(f"Best parameters for {model_name}: {clf.best_params_}")
    print(f"Training time for {model_name}: {end_time - start_time:.2f} seconds")

#Setup and run GridSearchCV specifically for the neural network
nn_params = {
    'lr': [0.1, 0.01, 0.001],
    'max_epochs': [10, 20, 30, 50, 100],
    'batch_size': [64, 128],
    'module__num_units': [100, 200],
    'module__activation': [nn.ReLU(), nn.Tanh()]
}



In [ ]:
#Define the PyTorch model
class SimpleNN(nn.Module):
    def __init__(self, num_features, num_units=128, activation=nn.ReLU()):
        super(SimpleNN, self).__init__()
        self.dense1 = nn.Linear(num_features, num_units)
        self.activation = activation
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(num_units, 1)

    def forward(self, X):
        X = self.activation(self.dense1(X))
        X = self.dropout(X)
        X = self.output(X)
        return X.squeeze()

#Neural network settings
net = NeuralNetClassifier(
    SimpleNN,
    module__num_features=3,
    criterion=nn.BCEWithLogitsLoss,
    optimizer=torch.optim.Adam,
    max_epochs=10,
    batch_size=64,
    device='cuda',
    iterator_train__shuffle=True
)

In [ ]:
#Evaluate over gridsearch best perfroming Neural Network.  
start_time = time.time()
gs_nn = GridSearchCV(net, nn_params, refit=True, cv=5, scoring='accuracy', verbose=0)
gs_nn.fit(X_train_umap, y_train)
end_time = time.time()
best_models['PyTorch NN'] = {
    'model': gs_nn.best_estimator_,
    'score': gs_nn.best_score_,
    'best_params': gs_nn.best_params_,
    'training_time': end_time - start_time
}
print("PyTorch NN: Best CV score - {:.9f}".format(gs_nn.best_score_))
print("Best parameters for PyTorch NN: ", gs_nn.best_params_)
print(f"Training time for PyTorch NN: {end_time - start_time:.2f} seconds")


In [ ]:
#Evaluate all models on the test set
test_scores = {}
for model_name, info in best_models.items():
    test_score = info['model'].score(X_test_umap, y_test)
    test_scores[model_name] = test_score
    print(f"Test Score for {model_name}: {test_score:.9f}")


In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
model_names = list(best_models.keys())
scores = [best_models[m]['score'] for m in model_names]
bar_height = 0.4
ax.barh(model_names, scores, height=bar_height, color='skyblue')
ax.set_xlabel('Best Cross-Validation Accuracy')
ax.set_title('Model Comparison')
plt.show()

fig, ax = plt.subplots(figsize=(10, 4))
scores = [test_scores[m] for m in model_names]
ax.barh(model_names, scores, height=bar_height, color='skyblue')
ax.set_xlabel('Accuracy on Test Set')
ax.set_title('Model Performance Comparison on Test Data')
plt.show()

In [ ]:
#Evaluate all models on the test set and generate additional metrics of interest:  Precision and Recall
test_scores = {}
additional_metrics = {}
for model_name, info in best_models.items():
    model = info['model']
    y_pred = model.predict(X_test_umap)
    test_scores[model_name] = model.score(X_test_umap, y_test)
    #y_prob = model.decision_function(X_test_umap)
    #auc_score = roc_auc_score(y_test, y_prob)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    additional_metrics[model_name] = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1#,
        #'auc': auc_score
    }
    print(f"Test Score for {model_name}: {test_scores[model_name]:.9f}")
    print(f"Precision for {model_name}: {precision:.9f}")
    print(f"Recall for {model_name}: {recall:.9f}")
    print(f"F1 Score for {model_name}: {f1:.9f}")
    #print(f"AUC Score for {model_name}: {auc_score:.9f}")

In [ ]:
#Generate Feature Importances
feature_importances = {}
for model_name, model_info in best_models.items():
    model = model_info['model']
    if hasattr(model, 'feature_importances_'):
        feature_importances[model_name] = model.feature_importances_
    elif hasattr(model, 'coef_'):
        feature_importances[model_name] = np.abs(model.coef_)

# Neural Network Permutation Importance
nn_model = best_models['PyTorch NN']['model']
perm = permutation_importance(nn_model, X_test_umap, y_test, n_repeats=5, random_state=42)
feature_importances['PyTorch NN'] = perm.importances_mean

for model_name, importances in feature_importances.items():
    print(f"Feature importances for {model_name}: {importances}")